In [5]:
import numpy as np
import matplotlib.pyplot as plt

from keras import models
from keras.layers import Dense, Conv1D, Reshape, Flatten, Lambda
from keras import backend as K

위에 임포트한 모듈 중에서 새로 보이는 것이 Lambda 입니다.

아래는 책 3분 딥러닝에 나오는 GAN 모델에 대한 클래스의 코드인데, 바로 이해하기 어렵습니다.

처음 보는 함수나 기능들은 실제로 타이핑을 해보면서 이해해보겠습니다.

In [6]:
class Data:
    def __init__(self, mu, sigma, ni_D):
        self.real_sample = lambda n_batch: np.random.normal(mu, sigma, n_batch, ni_D)
        self.in_sample = lambda n_batch: np.random.rand(n_batch, ni_D)
        

위의 Data 클래스는 평균, 표준편차와 어떤 차원의 길이(ni_D)를 저장하고 n_batch 라는 다른 차원의 길이를 입력받으면 (n_batch, ni_D) 크기의 정규분포를 만드는 함수와,  같은 크기의 0에서 1사이의 균일 분포를 만드는 함수를 저장하고 있습니다.

In [4]:
# GAN 모델링
def add_decorate(x):
    m = K.mean(x, axis=-1, keepdims=True)
    d = K.square(x - m)
    return K.concatenate([x, d], axis=-1)

def add_decorate_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2
    shape[1] *= 2
    return tuple(shape)
    
class GAN:
    def __init__(self, ni_D, nh_D, nh_G):
        self.ni_D = ni_D
        self.nh_D = nh_D
        self.nh_G = nh_G
        
        self.D = self.gen_D()
        self.G = self.gen_G()
        self.GD = self.make_GD()
        
    def gen_D(self):
        ni_D = self.ni_D
        nh_D = self.nh_D
        D = models.Sequential()
        D.add(Lambda(add_decorate, output_shape=add_decorate_shape, input_shape=(ni_D,)))
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(1, activation='sigmoid'))
        
        D.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return D
    
    def gen_G(self):
        ni_D = self.ni_D
        nh_D = self.nh_D
        
        G = models.Sequential()
        G.add(Reshape((ni_D, 1), input_shape=(ni_D,)))
        G.add(Conv1D(nh_G, 1, activation='relu'))
        G.add(Conv1D(nh_G, 1, activation='sigmoid'))
        G.add(Conv1D(1, 1))
        G.add(Flatten())
        
        G.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return G
    
    def make_GD(self):
        G, D = self.G, self.D
        GD = models.Sequential()
        GD.add(G)
        GD.add(D)
        D.trainable = False
        GD.G.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        D.trainable = True
        return GD
    
    def D_train_on_batch(self, Real, Gen):
        D = self.D
        X = np.concatenate([Real, Gen], axis=0)
        y = np.array([1]*Real.shape[0] + [0]*Gen.shape[0])
        D.train_on_batch(X, y)
        
    def GD_train_on_batch(self, Z):
        GD = self.GD
        y = np.array([1]*Z.shape[0])
        GD.train_on_batch(Z, y)
        
        